In [205]:
import pandas as pd
import numpy as np
from plotnine import *
import missingno as msno

import sys
print(sys.version)
print(pd.__version__)
print(np.__version__)
print(msno.__version__)

3.6.5 |Anaconda, Inc.| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
0.23.0
1.14.3
0.4.1


In [216]:
# 판다스를 통해 데이터를 로드해 온다.
# 여기에서는 캐글의 타이타닉 데이터를 사용한다. 
# 데이터 다운로드 : https://www.kaggle.com/c/titanic/data

train = pd.read_csv('/Users/heebunny/Documents/dataitgirls/all/train.csv')
train.shape

(891, 12)

In [217]:
test = pd.read_csv('/Users/heebunny/Documents/dataitgirls/all/test.csv')
test.shape

(418, 11)

In [218]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [219]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [221]:
# from sklearn.preprocessing import MinMaxScaler

# def normalize_age(data):
#     scaler = MinMaxScaler()
#     data["Age"] = scaler.fit_transform(data["Age"].values.reshape(-1,1))
#     return data


# def normalize_fare(data):
#     scaler = MinMaxScaler()
#     data["Fare"] = scaler.fit_transform(data["Fare"].values.reshape(-1,1))
#     return data

# train_ = normalize_age(train)
# test_ = normalize_age(test)

# train__ = normalize_fare(train)
# test__ = normalize_fare(test)

# train__.head()

In [222]:
train['Embarked'].fillna(train['Embarked'].mode()[0], inplace=True)

In [223]:
test['Embarked'].fillna(test['Embarked'].mode()[0], inplace=True)

In [224]:
train_cp = train.copy()
test_cp = test.copy()

In [225]:
# 결측치가 있는 지 확인
train['Name'].isnull().sum()

0

In [226]:
train['Name'].str.split(", ", expand=True).head()

,0,1
0,Braund,Mr. Owen Harris
1,Cumings,Mrs. John Bradley (Florence Briggs Thayer)
2,Heikkinen,Miss. Laina
3,Futrelle,Mrs. Jacques Heath (Lily May Peel)
4,Allen,Mr. William Henry


In [227]:
# 이름에서 Title 항목만 떼어낸다.
# 콤마로 분리를 하고 분리된 두 번째 데이터의 첫번째 항목을 가져온다.
train['Title'] = train['Name'].str.split(
    ", ", expand=True)[1].str.split(".", expand=True)[0]
train['Title'].head()

test['Title'] = test['Name'].str.split(
    ", ", expand=True)[1].str.split(".", expand=True)[0]
test['Title'].head()

0     Mr
1    Mrs
2     Mr
3     Mr
4    Mrs
Name: Title, dtype: object

In [228]:
train.query("Title in ('Dona', 'Lady', 'the Countess')")

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
556,557,1,1,"Duff Gordon, Lady. (Lucille Christiana Sutherl...",female,48.0,1,0,11755,39.6,A16,C,Lady
759,760,1,1,"Rothes, the Countess. of (Lucy Noel Martha Dye...",female,33.0,0,0,110152,86.5,B77,S,the Countess


In [229]:
# 결혼한 여성 관련 호칭은 Mrs로 분류
train.loc[train['Title'].isin(['Dona', 'Lady', 'the Countess', 'Mme', 'Mlle']), 'Title'] = 'Ms'
test.loc[test['Title'].isin(['Dona', 'Lady', 'the Countess', 'Mme', 'Mlle']), 'Title'] = 'Ms'
train['Title'].value_counts()

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Ms            6
Rev           6
Col           2
Major         2
Don           1
Jonkheer      1
Sir           1
Capt          1
Name: Title, dtype: int64

In [230]:
train.loc[train['Title'].isin(['Don', 'Sir', 'Capt', 'Col', 'Major', 'Master', 'Rev', 'Dr', 'Jonkheer']), 'Title'] = 'Mr'
test.loc[test['Title'].isin(['Don', 'Sir', 'Capt', 'Col', 'Major', 'Master', 'Rev', 'Dr', 'Jonkheer']), 'Title'] = 'Mr'
train['Title'].value_counts()

Mr      578
Miss    182
Mrs     125
Ms        6
Name: Title, dtype: int64

In [231]:
train.Age.isnull().sum()

177

In [232]:
train_groupby = train.copy()
test_groupby = test.copy()

In [233]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
Title            0
dtype: int64

In [239]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
Title            0
dtype: int64

In [240]:
test['Fare'].fillna(test['Fare'].mean(), inplace=True)

In [241]:
train_groupby['Age'].fillna(train.groupby('Title')['Age'].transform('median'), inplace=True)
test_groupby['Age'].fillna(test.groupby('Title')['Age'].transform('median'), inplace=True)

In [242]:
print(train.Age.isnull().sum())
print(train_groupby.Age.isnull().sum())

0
0


In [243]:
train = train_groupby
test = test_groupby

In [244]:
from sklearn.preprocessing import MinMaxScaler

def normalize_age(data):
    scaler = MinMaxScaler()
    data["Age"] = scaler.fit_transform(data["Age"].values.reshape(-1,1))
    return data


def normalize_fare(data):
    scaler = MinMaxScaler()
    data["Fare"] = scaler.fit_transform(data["Fare"].values.reshape(-1,1))
    return data

train = normalize_age(train)
test = normalize_age(test)

train = normalize_fare(train)
test = normalize_fare(test)

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,0.271174,1,0,A/5 21171,0.014151,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,0.472229,1,0,PC 17599,0.139136,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,0.321438,0,0,STON/O2. 3101282,0.015469,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,0.434531,1,0,113803,0.103644,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,0.434531,0,0,373450,0.015713,NaN,S,Mr


In [245]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Title'],
      dtype='object')

In [246]:
test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'Title'],
      dtype='object')

In [247]:
train['AgeBin'] = pd.cut(train['Age'].astype(int), 30)
test['AgeBin'] = pd.cut(test['Age'].astype(int), 30)

In [248]:
# 기계가 데이터를 이해할 수 있도록 
# 카테고리 데이터를 one-hot-encoding 해준다.
def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis=1)
        data = data.drop(column, axis=1)
    return data


dummy_columns = ["Sex", "Pclass", "Embarked", "AgeBin"]
train_dummy = dummy_data(train, dummy_columns)
test_dummy = dummy_data(test, dummy_columns)

print('원핫인코딩 전 shape')
print(train.shape)
print(test.shape)

print('get_dummies로 원핫인코딩 후 shape')
print(train_dummy.shape)
print(test_dummy.shape)

원핫인코딩 전 shape
(891, 14)
(418, 13)
get_dummies로 원핫인코딩 후 shape
(891, 48)
(418, 47)


In [249]:
train_dummy.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Title,...,"AgeBin_(0.667, 0.7]","AgeBin_(0.7, 0.733]","AgeBin_(0.733, 0.767]","AgeBin_(0.767, 0.8]","AgeBin_(0.8, 0.833]","AgeBin_(0.833, 0.867]","AgeBin_(0.867, 0.9]","AgeBin_(0.9, 0.933]","AgeBin_(0.933, 0.967]","AgeBin_(0.967, 1.0]"
0,1,0,"Braund, Mr. Owen Harris",0.271174,1,0,A/5 21171,0.014151,NaN,Mr,...,0,0,0,0,0,0,0,0,0,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0.472229,1,0,PC 17599,0.139136,C85,Mrs,...,0,0,0,0,0,0,0,0,0,0
2,3,1,"Heikkinen, Miss. Laina",0.321438,0,0,STON/O2. 3101282,0.015469,NaN,Miss,...,0,0,0,0,0,0,0,0,0,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0.434531,1,0,113803,0.103644,C123,Mrs,...,0,0,0,0,0,0,0,0,0,0
4,5,0,"Allen, Mr. William Henry",0.434531,0,0,373450,0.015713,NaN,Mr,...,0,0,0,0,0,0,0,0,0,0


In [250]:
# 사용하지 않을 컬럼을 제거해 피처로 사용할 컬럼만 남겨둔다.
def drop_not_concerned(data, columns):
    return data.drop(columns, axis=1)

not_concerned_columns = ["PassengerId", "Age", "Name", "Ticket", "Fare", "Title", "Cabin"]
X_train = drop_not_concerned(train_dummy, not_concerned_columns)
X_train = X_train.drop('Survived', axis=1)
X_test = drop_not_concerned(test_dummy, not_concerned_columns)

In [251]:
y_label = train['Survived']

In [252]:
X_test.head()

,SibSp,Parch,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,...,"AgeBin_(0.667, 0.7]","AgeBin_(0.7, 0.733]","AgeBin_(0.733, 0.767]","AgeBin_(0.767, 0.8]","AgeBin_(0.8, 0.833]","AgeBin_(0.833, 0.867]","AgeBin_(0.867, 0.9]","AgeBin_(0.9, 0.933]","AgeBin_(0.933, 0.967]","AgeBin_(0.967, 1.0]"
0,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [253]:
from sklearn.tree import DecisionTreeClassifier


In [254]:
model = DecisionTreeClassifier(max_depth=10, random_state=2018)
# model.fit(X_train, y_train)
# prediction = model.predict(X_test)

In [255]:
model.fit(X_train, y_label)
prediction = model.predict(X_test)

In [256]:
score = round(model.score(X_train, y_label)*100, 2)
score

83.84

In [257]:
# #결정트리 시각화 하기

# from sklearn.tree import export_graphviz
# import graphviz

In [258]:
# export_graphviz(model,
#                 feature_names=feature_names,
#                 class_names=["Perish", "Survived"],
#                 out_file="decision-tree.dot")

# with open("decision-tree.dot") as f:
#     dot_graph = f.read()
    
# graphviz.Source(dot_graph)

In [259]:
test['Survived'] = prediction
test.columns

submissions = test[['PassengerId', 'Survived']]
submissions.head()

submissions.to_csv('/Users/heebunny/Documents/dataitgirls/all/submission180827_3.csv', index=False)

In [260]:
submissions.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [261]:
submissions.shape

(418, 2)